## Домашнее задание по NLP # 2 [100 баллов]

#### Составление словарей для классификации по тональности

1) Разбейте всю коллекцию отзывов на предложения. 

2) Лемматизируйте все слова.

3) Обучите по коллекции предложений word2vec

4) Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    * тест на определение ближайших слов
    * тест на аналогии (мужчина – король : женщина – королева)
    * тест на определение лишнего слова

5) Постройте несколько визуализаций:
    1. TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    2. задайте координаты для нового пространства следующим образом:

- одна ось описывает отношение "плохо – хорошо", 
- вторая – "медленно – быстро" 
    
    3. найдите координаты названий банков в этих координатах.

In [1]:
import json
import bz2
import regex
from tqdm import tqdm
import pandas as pd

In [2]:
responses = []
with open('banki_responses.json', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [00:28, 7094.36it/s]


In [3]:
data = pd.DataFrame.from_dict(responses)
data = data.dropna()

In [4]:
data.head(2)

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
14,г. Москва,False,В Шоке,0,лицензия № 2673,yura1189,Тинькофф Банк,2015-06-05 13:52:40,"Также как и многие давний клиент банка, был до...",2.0
19,г. Москва,False,Некомпетентность сотрудников банка,2,лицензия № 1481,agelun,Сбербанк России,2015-06-05 15:00:54,Открыт вклад и счет в USD. Плюс к этому есть з...,1.0


In [5]:
positive = data[data.rating_grade == 5]
negative = data[data.rating_grade == 1]

In [6]:
comments_pos = list(positive.text)
comments_neg = list(negative.text)

In [7]:
len(comments_pos)

10511

In [80]:
import pymorphy2
import re

m = pymorphy2.MorphAnalyzer()

regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        text = re.sub(r'\s+', ' ', text)
        return regex.findall(text)
    except:
        return []

def lemmatize(text, pymorphy=m):
    try:
        return [pymorphy.parse(w.lower())[0].normal_form for w in text]
    except:
        return " "

In [82]:
words_only(sent_tokenize(comments_pos[0])[5])

['Жаль',
 'что',
 'прикрыли',
 'прием',
 'вкладов',
 'в',
 'офисе',
 'Озерки',
 'доступность',
 'офиса',
 'для',
 'меня']

In [73]:
sent_tokenize(comments_pos[0])

['Досрочно закрывали вклады перед отпуском.',
 'Жена заказывала заранее, но забыла заказать на меня.',
 'Сотрудник банка (СПБ Ленсовета 89) Наталья не отказала мне, хотя сумма значительная.',
 'Работает быстро, профессионально.',
 'Ничего лишнего.',
 'Жаль, что прикрыли прием вкладов в офисе Озерки - доступность офиса для меня.',
 'Заметил, что и несут деньги во вклады и берут кредиты.',
 'Желаю успехов банку.']

In [83]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

def clean_text(comments, remove_stopwords=True):
    
    sentences = []

    for i in tqdm(comments, total = len(comments)):

        sents = sent_tokenize(i)

        for ii in sents:

            sents_w = words_only(ii)

            if remove_stopwords:
                stops = stopwords.words("russian")
                sents_w = [w for w in sents_w if not w in stops]
            
            sents_w_l = lemmatize(sents_w)
            sentences.append(sents_w_l)

    return sentences

In [260]:
pos = clean_text(comments_pos)

In [149]:
neg = clean_text(comments_neg)

100%|██████████| 39855/39855 [22:48<00:00, 29.12it/s]


In [150]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(
    min_count=10,
    window=2,
    size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=0)

w2v_model_neg = Word2Vec(
    min_count=10,
    window=2,
    size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=0)

In [160]:
w2v_model.build_vocab(pos)

In [151]:
w2v_model_neg.build_vocab(neg)

In [161]:
w2v_model.train(pos, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(22972243, 50137950)

In [153]:
w2v_model_neg.train(neg, total_examples=w2v_model_neg.corpus_count, epochs=30, report_delay=1)

(101549345, 219986100)

In [177]:
vocabulary = w2v_model.wv.vocab
list(vocabulary.keys())[:10]

['досрочно',
 'закрывать',
 'вклад',
 'отпуск',
 'жена',
 'заказывать',
 'заранее',
 'забыть',
 'заказать',
 'сотрудник']

#### Тест на ближайшие слова

In [90]:
w2v_model.wv.most_similar(positive=["сбербанк"])

[('сбер', 0.5389305353164673),
 ('втб', 0.4748550057411194),
 ('мкб', 0.4336012601852417),
 ('впб', 0.4152097702026367),
 ('ситибанк', 0.4122902452945709),
 ('райффайзенбанк', 0.4120444059371948),
 ('росбанк', 0.40170425176620483),
 ('райфайзный', 0.3979722857475281),
 ('возрождение', 0.39185988903045654),
 ('уралсиб', 0.38991856575012207)]

In [91]:
w2v_model.wv.most_similar(positive=["кредит"])

[('автокредит', 0.6853684186935425),
 ('ипотека', 0.6027327179908752),
 ('рассрочка', 0.5901181697845459),
 ('потреба', 0.5434262156486511),
 ('рефинансировать', 0.5136533975601196),
 ('потребительский', 0.5044800043106079),
 ('заём', 0.47963714599609375),
 ('переплата', 0.46628332138061523),
 ('перекредитование', 0.4644802510738373),
 ('кредитование', 0.45030754804611206)]

#### Тест на аналогии

In [111]:
import wget
url = 'https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt'
wget.download(url)


'ru_analogy_tagged (2).txt'

In [117]:
res = w2v_model.wv.evaluate_word_analogies('ru_analogy_tagged.txt')

In [115]:
res

(None,
 [{'section': 'capital-common-countries', 'correct': [], 'incorrect': []},
  {'section': 'capital-world', 'correct': [], 'incorrect': []},
  {'section': 'currency', 'correct': [], 'incorrect': []},
  {'section': 'city-in-state', 'correct': [], 'incorrect': []},
  {'section': 'family', 'correct': [], 'incorrect': []},
  {'section': 'gram1-Aective-to-adverb', 'correct': [], 'incorrect': []},
  {'section': 'gram2-opposite', 'correct': [], 'incorrect': []},
  {'section': 'gram6-nationality-Aective', 'correct': [], 'incorrect': []},
  {'section': 'Total accuracy', 'correct': [], 'incorrect': []}])

In [178]:
w2v_model.wv.similarity('превосходно', 'ужасный')

-0.064660884

In [183]:
w2v_model.wv.similarity('быстро', 'сонный')

0.024014438

In [284]:
w2v_model.wv.most_similar(positive=['отлично', 'оперативно'], negative=['квалифицированно'])

[('твёрдый', 0.3437631130218506),
 ('ку', 0.3397331237792969),
 ('быстро', 0.32683467864990234),
 ('масло', 0.31971922516822815),
 ('известность', 0.3027058243751526),
 ('on', 0.30174827575683594),
 ('великолепно', 0.30017441511154175),
 ('успешно', 0.28761041164398193),
 ('блестяще', 0.284415066242218),
 ('ура', 0.2799515724182129)]

In [206]:
w2v_model_neg.wv.most_similar(positive=['ужасный'])

[('отвратительный', 0.6336706876754761),
 ('отвратный', 0.47922956943511963),
 ('безалаберный', 0.47199440002441406),
 ('безобразный', 0.4716470539569855),
 ('плохой', 0.463365763425827),
 ('неклиентоориентировать', 0.452890545129776),
 ('ужасающий', 0.44730687141418457),
 ('скотский', 0.43871551752090454),
 ('омерзительный', 0.41910219192504883),
 ('свинский', 0.40692266821861267)]

In [207]:
vec = (w2v_model["отлично"] - w2v_model["ужасно"]) * w2v_model["оперативно"]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [208]:
w2v_model.wv.most_similar(positive=[vec])

[('комитет', 0.21279466152191162),
 ('одобрение', 0.20015111565589905),
 ('одобрить', 0.19621658325195312),
 ('готовый', 0.19446590542793274),
 ('подъезжать', 0.19273491203784943),
 ('интервал', 0.1890474557876587),
 ('готовность', 0.18551817536354065),
 ('изготовить', 0.17940875887870789),
 ('привозить', 0.1768336296081543),
 ('й', 0.1738775223493576)]

In [145]:
vec[:10]

array([ 0.09369595,  0.0079945 ,  0.06040062, -0.08668521,  0.05974599,
       -0.03268576,  0.04211259,  0.00947612, -0.05109083, -0.23750074],
      dtype=float32)

#### Тест на определение лишнего слова

In [197]:
w2v_model.wv.doesnt_match('отлично великолепно ужасно'.split())

'ужасно'

In [205]:
w2v_model_neg.wv.doesnt_match('отвратительный круто ужасный'.split())

'круто'

#### Постройте несколько визуализаций:

1. TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов

In [223]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

def top_words(data, n_top=100):
    top_words = []

    fd = FreqDist()
    for s in tqdm(data):
        fd.update(s)

    for w in fd.most_common(n_top):
        top_words.append(w[0])

    top_words_vec = w2v_model.wv[top_words]
    
    return top_words, top_words_vec

In [302]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

def tsne_plot(data=pos, n_top=100, transform = False):

    if transform:
        top_words_vec = top_words_trans()
    else:
        top_words_vec = top_words(data=data, n_top=n_top)
    
    tsne = TSNE(n_components=2, random_state=0, init="pca")
    top_words_tsne = tsne.fit_transform(top_words_vec[1])

    p = figure(tools="pan,wheel_zoom,reset,save",
               toolbar_location="above",
               title="word2vec T-SNE (w2v_model, top100 words)")

    source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                        x2=top_words_tsne[:,1],
                                        names=top_words_vec[0]))

    p.scatter(x="x1", y="x2", size=8, source=source)

    labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                      text_font_size="8pt", text_color="#555555",
                      source=source, text_align='center')
    p.add_layout(labels)

    show(p)

Loading BokehJS ...

In [298]:
# 100 из положительных постов
tsne_plot(pos, n_top=100)
# осмысленные кластеры: вокруг "сотрудник", вокруг "документ"

In [229]:
# 100 из негативных постов
tsne_plot(neg, n_top=100)
# осмысленные кластеры: вокруг "вопрос", вокруг "обратиться", вокруг "счет", вокруг "неделя"

#### Постройте несколько визуализаций:

    2. задайте координаты для нового пространства следующим образом: одна ось - "плохо – хорошо", вторая – "медленно – быстро"
    3. найдите координаты названий банков в этих координатах.
      
*Вектор слова "хорошо" - вектор слова "плохо" = разница между хорошими и плохими словами.*

*Вектор слова "сбербанк" * новый вектор = координата по первой оси.* 

*Аналогично – для второй оси.*

In [261]:
len(pos), len(neg)

(186256, 807073)

In [262]:
import random as rand
rand.shuffle(neg)
neg_short = neg[:186000]

In [263]:
pos_neg = pos.copy()

for i in neg_short:
    pos_neg.append(i)

In [265]:
len(pos_neg)

372256

In [266]:
w2v_model_pos_neg = Word2Vec(
    min_count=10,
    window=2,
    size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=0)

w2v_model_pos_neg.build_vocab(pos_neg)
w2v_model_pos_neg.train(pos_neg, total_examples=w2v_model_pos_neg.corpus_count, epochs=30, report_delay=1)

(47295844, 100787520)

In [281]:
w2v_model_pos_neg.wv.most_similar(positive=['медленно'], topn=10)

[('вяло', 0.5216045379638672),
 ('нервно', 0.4690588712692261),
 ('неспешно', 0.46819913387298584),
 ('садиться', 0.441694438457489),
 ('тормозить', 0.4303429424762726),
 ('шустро', 0.4228843152523041),
 ('закипать', 0.4202459156513214),
 ('рассосаться', 0.4122873544692993),
 ('позитивно', 0.4117111265659332),
 ('сладить', 0.4030282497406006)]

In [274]:
list_bank = ['сбербанк', 'сбер', 'sberbank', 'втб', 'vtb', \
             'райф', 'райфайзен', 'райффайзен', 'райффайзенбанк', 'raiffeisen', \
             'ситибанк', 'citibank', 'росбанк', 'промсвязь', 'промсвязьбанк', 'собинбанк', \
             'возрождение', 'авангард', 'бинбанк', 'уралсиб', 'альфа', 'alfabank', 'транскредитбанк']

In [323]:
words_vec_x1 = w2v_model_pos_neg.wv['хорошо'] - w2v_model_pos_neg.wv['плохо']
words_vec_x2 = w2v_model_pos_neg.wv['быстро'] - w2v_model_pos_neg.wv['медленно']

In [324]:
proj = pd.DataFrame([words_vec_x1, words_vec_x2], index=["height", "second"]).transpose()

In [356]:
df_plot = w2v_model_pos_neg.wv[list_bank].dot(proj)
df_plot = pd.DataFrame(df_plot)
df_plot['bank'] = list_bank
df_plot.head(2)

,0,1,bank
0,-7.993644,0.413735,сбербанк
1,-11.056973,-5.753331,сбер


In [364]:
w2v_model_pos_neg.wv['сбер'].dot(words_vec_x2)

-5.75333

In [366]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="банки")

source = ColumnDataSource(data=dict(x1=df_plot[0],
                                    x2=df_plot[1],
                                    names=df_plot['bank']))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)
p.xaxis.axis_label = 'Плохо --> Хорошо'
p.yaxis.axis_label = 'Медленно --> Быстро'

show(p)